In [1]:
import os
from dotenv import load_dotenv
# import beaapi
import json
import pandas as pd
import requests

In [2]:
load_dotenv()
beakey = os.environ.get('API_KEY_BEA')

In [3]:
method = "GetData"
data_set = "Regional"
params = 'TableName=SAINC1&GeoFips=STATE&LineCode=3&Year=2022'

In [4]:
data = requests.get(
        f'https://apps.bea.gov/api/data?&UserID={beakey}&method={method}&datasetname={data_set}&{params}&ResultFormat=JSON'
    ).json()

In [7]:
df_data = pd.DataFrame(data["BEAAPI"]['Results']['Data'])
df_data = df_data.set_index('GeoName')
df_data = df_data.sort_values('DataValue', ascending=False)
df_data.head()

,Code,GeoFips,TimePeriod,CL_UNIT,UNIT_MULT,DataValue,NoteRef
GeoName,,,,,,,
District of Columbia,SAINC1-3,11000,2022,Dollars,0,96092,2
Massachusetts,SAINC1-3,25000,2022,Dollars,0,84551,2
Connecticut,SAINC1-3,09000,2022,Dollars,0,83340,2
New England,SAINC1-3,91000,2022,Dollars,0,78603,2
New Jersey,SAINC1-3,34000,2022,Dollars,0,77206,2
